In [29]:
import pandas as pd
import numpy as np
from copy import deepcopy
import matplotlib.pyplot as plt
from scipy.signal import hilbert, stft

# Clase Info 

In [ ]:
class Info():
    """Clase para almacenar información acerca del registro de datos.
    Esta clase se comporta como un diccionario.
    """

    def __init__(self, ch_names=None, ch_types="unknown", bads=None, sfreq=512,
                 description=None, experimenter="No data", subject_info=None):
        """
        Genera un objeto Info().

        Parameteros
        ----------
        ch_names : list of str, optional
            Lista con los nombres de los canales.
        ch_types : str or list of str, optional
            Tipo de cada canal (ej.: 'eeg', 'ecg', etc.) o un único tipo para todos.
        bads : list of str, optional
            Lista de canales marcados como “malos”.
        sfreq : float, optional
            Frecuencia de muestreo en Hz (por defecto 512).
        description : str, optional
            Descripción del registro de datos.
        experimenter : str, optional
            Nombre del experimentador.
        subject_info : dict, optional
            Información adicional del sujeto.

        Raises
        ------
        ValueError
            Si `ch_names` y `ch_types` son listas de distinta longitud.
        """
        # Validación de longitudes
        if isinstance(ch_types, (list, tuple)) and ch_names is not None: # Comprueba si la variable ch_types es una instancia de alguno de los tipos indicados.
            if len(ch_types) != len(ch_names):
                raise ValueError(
                    "Si 'ch_types' es lista, debe tener la misma longitud que 'ch_names'."
                )

        # Datos internos #  construcción del diccionario interno self._data donde guardamos todos los parámetros de Info de forma unificada.
        self._data = {
            'ch_names': list(ch_names) if ch_names is not None else [], # Crea la clave ch_names asignándole una copia de la lista ch_names cuando no es None, y una lista vacía [] en caso contrario.
            'ch_types': list(ch_types) if isinstance(ch_types, (list, tuple)) else ch_types, # # Si ch_types es lista/tupla, crea y almacena una copia de esa secuencia; en caso contrario (p.ej., un único string) guarda el valor directamente. 
            'bads': list(bads) if bads is not None else [], # Si `bads` (lista de canales malos) existe, crea una copia con `list(bads)`; si es `None`, inicializa como lista vacía.
            'sfreq': sfreq,
            'description': description,
            'experimenter': experimenter,
            'subject_info': dict(subject_info) if subject_info is not None else {} # Si `subject_info` es un dict, crea una copia con `dict(subject_info)`; si es `None`, inicializa como diccionario vacío.
        }

    def __contains__(self, key):
        """Permitе verificar si una clave está presente en el objeto."""
        return key in self._data # # Devuelve True si ‘key’ es una de las claves en el diccionario interno self._data; en caso contrario, False.

    def __getitem__(self, key):
        """
        Permite acceder a elementos como si fuera un diccionario.
        
        Raises
        ------
        KeyError
            Si la clave no existe en el objeto.
        """
        try:
            return self._data[key]
        except KeyError:
            raise KeyError(f"La clave '{key}' no existe en Info.")

    def __len__(self):
        """Devuelve la cantidad de elementos (claves) almacenados."""
        return len(self._data)

    def get(self, key, default=None):
        """
        Obtiene el valor de una clave específica, devolviendo `default`
        si la clave no está presente.
        """
        return self._data.get(key, default)

    def keys(self):
        """Devuelve un iterador con las claves del objeto."""
        return self._data.keys()

    def items(self):
        """Devuelve un iterador con los pares (clave, valor) almacenados."""
        return self._data.items()

    def rename_channels(self, mapping):
        """
        Permite renombrar canales de forma segura.

        Parameters
        ----------
        mapping : dict
            Diccionario que mapea nombres antiguos de canales a nombres nuevos.

        Raises
        ------
        ValueError
            Si alguna clave de mapping no existe en 'ch_names', o si
            los nuevos nombres introducen duplicados.
        """
        ch_names = self._data['ch_names']
        bads = self._data['bads']

        # Validar que todas las antiguas existen
        for old in mapping:
            if old not in ch_names:
                raise ValueError(f"No existe el canal '{old}' en ch_names.")

        # Generar lista nueva y comprobar duplicados
        new_ch_names = []
        for name in ch_names:
            new_ch_names.append(mapping.get(name, name))
        if len(new_ch_names) != len(set(new_ch_names)):
            raise ValueError("Renombrado produciría nombres duplicados en ch_names.")

        # Renombrar bads si corresponde
        new_bads = [mapping.get(b, b) for b in bads]

        # Aplicar cambios
        self._data['ch_names'] = new_ch_names
        self._data['bads'] = new_bads


### Prueba de la Clase Info

In [16]:
# 1) Definir canales y crear Info
canales = [f"C{i+1}" for i in range(3)]      # es una list comprehension de Python que genera de forma concisa la lista ['C1', 'C2', 'C3'].
                                             # range(3) produce un iterador de valores enteros 0, 1, 2.
                                             # el bucle implícito for i in range(3) recorre esos tres valores.
                                             # para cada i, la expresión f"C{i+1}" construye un f-string.
                                             # {i+1} evalúa el valor de i+1 y lo convierte a cadena, de modo que para i=0 produce "1", para i=1 "2", etc.
                                             # Cada valor resultante ("C1", "C2", "C3") se va añadiendo a la lista final.   
info = Info(
    ch_names=canales,
    ch_types=['eeg'] * 3,
    bads=['C2'],
    sfreq=512,
    description="Registro EEG para análisis de patrones ERDS",
    experimenter="MSc. BALDEZZARI Lucas",
    subject_info={'edad': 29, 'sexo': 'M'}
)

# 2) Acceso a datos
print(info['ch_names'])       # → ['C1', 'C2', 'C3']
print(info.get('sfreq'))      # → 512

# 3) Renombrar canales correctamente
mapping = {'C1': 'Fz', 'C2': 'Cz', 'C3': 'Pz'}
info.rename_channels(mapping)

# 4) Verificar resultados
print(info['ch_names'])       # → ['Fz', 'Cz', 'Pz']
print(info['bads'])           # → ['Cz']

# 5) Recorrer todas las claves y valores
for clave, valor in info.items():
    print(f"{clave}: {valor}")


['C1', 'C2', 'C3']
512
['Fz', 'Cz', 'Pz']
['Cz']
ch_names: ['Fz', 'Cz', 'Pz']
ch_types: ['eeg', 'eeg', 'eeg']
bads: ['Cz']
sfreq: 512
description: Registro EEG para análisis de patrones ERDS
experimenter: MSc. BALDEZZARI Lucas
subject_info: {'edad': 29, 'sexo': 'M'}


# Clase Anotaciones

In [32]:
class Anotaciones:
    """
    Clase para almacenar y gestionar anotaciones (eventos) en registros fisiológicos.
    Permite la adición, eliminación, búsqueda, recuperación y persistencia en CSV.
    """

    def __init__(self, annotations=None, onset=None, duration=None, description=None):
        """
        Inicializa la clase con los datos de las anotaciones.

        Parameters
        ----------
        annotations : list of dict or pandas.DataFrame, optional
            Lista de diccionarios con las anotaciones o
            un DataFrame ya construido.
        onset : list of float, optional
            Lista de tiempos de inicio de eventos (en segundos).
        duration : list of float, optional
            Lista de duraciones de eventos (en segundos).
        description : list of str, optional
            Lista de descripciones de eventos.

        Raises
        ------
        ValueError
            Si las listas onset, duration y description tienen longitudes distintas,
            o falta alguna de ellas cuando se pasa onset.
        """
        # Caso: se proporcionan listas onset/duration/description
        if annotations is None and onset is not None:
            if duration is None or description is None:
                raise ValueError("Debe proporcionar onset, duration y description juntos.")
            n = len(onset)
            if len(duration) != n or len(description) != n:
                raise ValueError(
                    "Las listas onset, duration y description deben tener la misma longitud."
                )
            self._df = pd.DataFrame({
                'onset': onset,
                'duration': duration,
                'description': description
            })
        # Caso: se proporciona un DataFrame
        elif isinstance(annotations, pd.DataFrame):
            self._df = annotations.copy()
        # Caso: lista de diccionarios
        elif isinstance(annotations, list):
            self._df = pd.DataFrame(annotations)
        # Caso: sin datos
        else:
            self._df = pd.DataFrame(columns=['onset', 'duration', 'description'])

        # Verificar columnas mínimas
        for col in ('onset', 'description'):
            if col not in self._df.columns:
                raise ValueError(
                    f"Falta la columna obligatoria '{col}' en las anotaciones"
                )
        # Asegurar columna duration
        if 'duration' not in self._df.columns:
            self._df['duration'] = pd.NA

        self._df.reset_index(drop=True, inplace=True)

    def add(self, onset, duration, description, **kwargs):
        """
        Agrega una nueva anotación.

        Parameters
        ----------
        onset : float
            Tiempo de inicio del evento (en segundos).
        duration : float
            Duración del evento (en segundos).
        description : str
            Descripción del evento.
        **kwargs : dict, optional
            Campos adicionales a incluir en la anotación.
        """
        nueva = {
            'onset': onset,
            'duration': duration,
            'description': description
        }
        nueva.update(kwargs)
        self._df = pd.concat([self._df, pd.DataFrame([nueva])], ignore_index=True)

    def remove(self, index=None, description=None):
        """
        Elimina una anotación por índice o descripción exacta.
        """
        if index is not None:
            if index in self._df.index:
                self._df.drop(index, inplace=True)
            else:
                raise ValueError(f"Índice {index} no existe en las anotaciones")
        elif description is not None:
            mask = self._df['description'] == description
            if not mask.any():
                raise ValueError(
                    f"No se encontró ninguna anotación con descripción '{description}'"
                )
            self._df = self._df.loc[~mask]
        else:
            raise ValueError("Debe especificar index o description para eliminar")

        self._df.reset_index(drop=True, inplace=True)

    def get_annotations(self):
        """
        Devuelve un DataFrame con todas las anotaciones.
        """
        return self._df.copy()

    def find(self, keyword):
        """
        Busca anotaciones cuya descripción contenga la keyword (case-insensitive).
        """
        mask = self._df['description'].str.contains(keyword, case=False, na=False)
        return self._df.loc[mask].copy()

    def save(self, filepath):
        """
        Guarda las anotaciones en un archivo CSV.
        """
        self._df.to_csv(filepath, index=False)

    def load(self, filepath):
        """
        Carga anotaciones desde un archivo CSV, reemplazando las existentes.
        """
        df = pd.read_csv(filepath)
        for col in ('onset', 'description'):
            if col not in df.columns:
                raise ValueError(f"El CSV no contiene la columna requerida '{col}'")
        if 'duration' not in df.columns:
            df['duration'] = pd.NA
        self._df = df.copy()
        self._df.reset_index(drop=True, inplace=True)


### Prueba de la Clase Anotaciones

In [33]:
# Instanciamos con los vectores que dio el profe
anotaciones = Anotaciones(
    onset=[5.0, 12.5, 20.0],
    duration=[2.0, 3.0, 3.5],
    description=["Inicio_Experimento", "Evento_1", "Evento_2"]
)

#anotaciones.save("mis_anotaciones.csv")

print(anotaciones.get_annotations())
#   onset  duration      description
#0    5.0       2.0  Inicio_Experimento
#1   12.5       3.0          Evento_1
#2   20.0       3.5          Evento_2

# Agregamos un nuevo evento
anotaciones.add(onset=30.0, duration=2.85, description="Evento_3")
print(anotaciones.get_annotations())

#Agregamos otro evento 
anotaciones.add(onset= 2.4, duration=3.12, description="Evento 4")

# Eliminamos por descripción
anotaciones.remove(description="Evento_1")
print(anotaciones.get_annotations())

# Buscamos con find()
anotaciones.add(onset=40.0, duration=3.1, description="Evento_3")
print(anotaciones.find("Evento_3"))
anotaciones.save("mis_anotaciones.csv")



   onset  duration         description
0    5.0       2.0  Inicio_Experimento
1   12.5       3.0            Evento_1
2   20.0       3.5            Evento_2
   onset  duration         description
0    5.0      2.00  Inicio_Experimento
1   12.5      3.00            Evento_1
2   20.0      3.50            Evento_2
3   30.0      2.85            Evento_3
   onset  duration         description
0    5.0      2.00  Inicio_Experimento
1   20.0      3.50            Evento_2
2   30.0      2.85            Evento_3
3    2.4      3.12            Evento 4
   onset  duration description
2   30.0      2.85    Evento_3
4   40.0      3.10    Evento_3


# Clase RawSignal

In [39]:
class RawSignal:
    """
    Clase para manejar señales fisiológicas en formato NumPy.

    Atributos mínimos:
      - data        : np.ndarray (n_canales x n_muestras)
      - sfreq       : float (Hz)
      - info        : Info (metadatos)
      - first_samp  : int (índice de la primera muestra válida)
      - anotaciones : Anotaciones (eventos asociados)
    """

    def __init__(self, data, sfreq, info=None, anotaciones=None, first_samp=0):
        # Verificar formato de data
        if not isinstance(data, np.ndarray) or data.ndim != 2:
            raise ValueError(
                        f"El array 'data' debe ser un np.ndarray 2D (canales x muestras), "
                        f"recibiendo forma {getattr(data, 'shape', None)}"
                            )
        self.data = data.copy()
        self.sfreq = float(sfreq)

        if info is None:
            # Crear Info básico con nombres genéricos
            ch_names = [f"Chan{i}" for i in range(data.shape[0])]
            self.info = Info(ch_names=ch_names,
                             ch_types=["unknown"] * data.shape[0],
                             sfreq=self.sfreq)
        else:
            self.info = info

        # Validar first_samp
        if not isinstance(first_samp, int) or not (0 <= first_samp < data.shape[1]):
            raise ValueError("'first_samp' debe estar en [0, n_muestras) y ser entero.")
        self.first_samp = first_samp

        # Anotaciones
        if anotaciones is None:
            self.anotaciones = Anotaciones()
        else:
            self.anotaciones = anotaciones

    def get_data(self, picks=None, start=0.0, stop=None, reject=None, times=False):
        """
        Obtiene muestras de la señal en un rango temporal dado.

        Parameters
        ----------
        picks   : list[str] o list[int], optional
                   Canales (nombres o índices) a extraer. Por defecto todos.
        start   : float, tiempo en segundos desde first_samp (default 0).
        stop    : float, tiempo en segundos hasta (default None = final).
        reject  : float, umbral pico a pico para filtrar canales (default None).
        times   : bool, si True retorna también vector de tiempos.

        Returns
        -------
        data_sel : np.ndarray, formas (n_canales_sel x n_muestras_sel)
        times_vec : np.ndarray, vector de tiempos (solo si times=True)
        """
        n_ch, n_samps = self.data.shape
        # Selección de canales
        if picks is None:
            idx_ch = np.arange(n_ch)
        else:
            # si nombres
            if all(isinstance(p, str) for p in picks):
                names = self.info['ch_names']
                idx_ch = [names.index(p) for p in picks]
            else:
                idx_ch = np.array(picks, dtype=int)
            # validar rango
            if any((i < 0 or i >= n_ch) for i in idx_ch):
                raise ValueError("Algunos índices de canal fuera de rango.")

        # Conversión de tiempos a muestras
        s0 = self.first_samp + int(start * self.sfreq)
        s1 = n_samps if stop is None else self.first_samp + int(stop * self.sfreq)
        if not (self.first_samp <= s0 < s1 <= n_samps):
            raise ValueError("Valores de start/stop fuera de rango de la señal.")

        # Extracción
        data_sel = self.data[idx_ch, s0:s1]

        # Rechazo por amplitud
        if reject is not None:
            ptp = data_sel.ptp(axis=1)
            mask = ptp <= reject
            data_sel = data_sel[mask, :]

        # Vector de tiempos
        if times:
            times_vec = (np.arange(s0, s1) - self.first_samp) / self.sfreq
            return data_sel, times_vec
        return data_sel

    def drop_channels(self, ch_names):
        """
        Elimina canales por nombre, devuelve nueva instancia RawSignal.
        """
        # Índices a eliminar
        all_names = self.info['ch_names']
        to_drop = [all_names.index(ch) for ch in ch_names]
        mask = [i for i in range(self.data.shape[0]) if i not in to_drop]

        # Nuevo data y nueva info
        new_data = self.data[mask, :]
        new_info = deepcopy(self.info)
        new_info._data['ch_names'] = [all_names[i] for i in mask]
        new_info._data['ch_types'] = [new_info._data['ch_types'][i] for i in mask]

        # Anotaciones se conservan
        return RawSignal(new_data, self.sfreq, info=new_info,
                         anotaciones=deepcopy(self.anotaciones),
                         first_samp=self.first_samp)

    def crop(self, tmin=0.0, tmax=None):
        """
        Recorta la señal entre tmin y tmax y ajusta first_samp y anotaciones.
        """
        n_ch, n_samps = self.data.shape
        start_idx = self.first_samp + int(tmin * self.sfreq)
        end_idx = n_samps if tmax is None else self.first_samp + int(tmax * self.sfreq)
        if not (self.first_samp <= start_idx < end_idx <= n_samps):
            raise ValueError("tmin/tmax fuera de rango.")

        # Extraer segmento
        new_data = self.data[:, start_idx:end_idx]

        # Ajustar annotations
        df = self.anotaciones.get_annotations()
        # filtramos eventos dentro [tmin, tmax)
        df2 = df[(df['onset'] >= tmin) & (df['onset'] < (tmax or df['onset'].max()))].copy()
        # reescalamos onset restando tmin
        df2['onset'] = df2['onset'] - tmin
        new_anots = Anotaciones(annotations=df2)

        # Info se mantiene (first_samp adaptada)
        return RawSignal(new_data, self.sfreq, info=deepcopy(self.info),
                         anotaciones=new_anots, first_samp=0)
        
    def describe(self):
        names = self.info['ch_names']
        types = self.info['ch_types']
        stats = []
        for i in range(self.data.shape[0]):
            channel = self.data[i, :]
            vals = {'name': names[i],
                    'type': types[i],
                    'min': float(channel.min()),
                    'Q1': float(np.percentile(channel, 25)),
                    'mediana': float(np.percentile(channel, 50)),
                    'Q3': float(np.percentile(channel, 75)),
                    'max': float(channel.max())}
            stats.append(vals)
        import pandas as pd
        return pd.DataFrame(stats)
    
    def filter(self, l_freq, h_freq, notch_freq=50.0, order=4, fir_window='hamming'):
        # Validar parámetros
        if l_freq <= 0 or h_freq <= 0 or l_freq >= h_freq or h_freq >= self.sfreq / 2:
            raise ValueError("Frecuencias l_freq y h_freq no válidas.")
        if notch_freq <= 0:
            raise ValueError("notch_freq debe ser positivo.")

        # Diseño filtro pasabanda IIR Butterworth
        nyq = 0.5 * self.sfreq
        low = l_freq / nyq
        high = h_freq / nyq
        b, a = butter(order, [low, high], btype='band')
        filtered = filtfilt(b, a, self.data, axis=1)

        # Diseño notch
        q = 30.0
        b_n, a_n = iirnotch(notch_freq / nyq, q)
        filtered = filtfilt(b_n, a_n, filtered, axis=1)

        return RawSignal(filtered, self.sfreq, info=deepcopy(self.info),
                         anotaciones=deepcopy(self.anotaciones),
                         first_samp=self.first_samp)


    def pick(self, picks):
        n_ch = self.data.shape[0]
        all_names = self.info['ch_names']
        if isinstance(picks, str):
            if picks not in all_names:
                raise ValueError(f"Canal '{picks}' no existe.")
            sel = [all_names.index(picks)]
        elif isinstance(picks, slice):
            sel = list(range(n_ch))[picks]
        else:
            sel = []
            for p in picks:
                if isinstance(p, str):
                    if p not in all_names:
                        raise ValueError(f"Canal '{p}' no existe.")
                    sel.append(all_names.index(p))
                else:
                    i = int(p)
                    if i < 0 or i >= n_ch:
                        raise ValueError(f"Índice de canal {i} fuera de rango.")
                    sel.append(i)

        new_data = self.data[sel, :]
        new_info = deepcopy(self.info)
        chn = []
        tp = []
        for i in sel:
            chn.append(new_info._data['ch_names'][i])
            tp.append(new_info._data['ch_types'][i])
        new_info._data['ch_names'] = chn
        new_info._data['ch_types'] = tp

        return RawSignal(new_data, self.sfreq, info=new_info,
                         anotaciones=deepcopy(self.anotaciones),
                         first_samp=self.first_samp)
    
    def set_anotaciones(self, anotaciones):
        """
        Asocia un objeto de tipo 'Anotaciones' a la señal fisiológica.

        Parameters
        ----------
        anotaciones : Anotaciones
            Objeto de la clase 'Anotaciones'.

        Raises
        ------
        TypeError
            Si 'anotaciones' no es instancia de Anotaciones.
        ValueError
            Si alguna anotación está fuera del rango temporal de la señal.
        """
        if not isinstance(anotaciones, Anotaciones):
            raise TypeError("El parámetro debe ser una instancia de Anotaciones.")
        df = anotaciones.get_annotations()
        # Verificar que onset esté dentro de duración de la señal
        max_time = (self.data.shape[1] - self.first_samp) / self.sfreq
        for _, row in df.iterrows():
            if row['onset'] < 0 or row['onset'] > max_time:
                raise ValueError(
                    f"La anotación en {row['onset']}s está fuera del rango [0, {max_time}]"
                )
        self.anotaciones = anotaciones
        
    def plot(self, picks=None, start=0.0, duration=10.0, show_anotaciones=True):
        # Extraer segmento
        stop = start + duration
        if show_anotaciones:
            data_sel, times_vec = self.get_data(picks=picks,
                                               start=start,
                                               stop=stop,
                                               times=True)
        else:
            data_sel = self.get_data(picks=picks,
                                     start=start,
                                     stop=stop,
                                     times=False)
            times_vec = (np.arange(
                int(start*self.sfreq),
                int(stop*self.sfreq)
            ) / self.sfreq)

        # Trazar
        plt.figure()
        for idx in range(data_sel.shape[0]):
            plt.plot(times_vec, data_sel[idx, :], label=self.info['ch_names'][idx])
        plt.xlabel('Time (s)')
        plt.ylabel('Amplitude')
        plt.title('RawSignal')
        plt.legend()
#falta implementar __getitem__()

### Prueba Clase RawDignal

In [38]:
# 1. Datos de prueba: 4 canales, 100 muestras
data = np.arange(4 * 100).reshape(4, 100).astype(float)
sfreq = 50.0  # Hz

# 2. Instanciar RawSignal
rs = RawSignal(data, sfreq)

# 3. Probar get_data
d_all = rs.get_data()  # Todos los datos
d_segment, times = rs.get_data(start=0.5, stop=1.5, times=True)  # 0.5s a 1.5s

# 4. Probar describe
df_stats = rs.describe()

# 5. Probar drop_channels
rs_drop = rs.drop_channels(['Chan1', 'Chan3'])
drop_names = rs_drop.info['ch_names']

# 6. Probar crop
rs_crop = rs.crop(tmin=1.0, tmax=2.0)
crop_shape = rs_crop.data.shape

# 7. Probar pick
rs_pick = rs.pick(['Chan0', 'Chan2'])
pick_names = rs_pick.info['ch_names']

# 8. Mostrar resultados

print("=== Datos del segmento (0.5s–1.5s) ===")
df_segment = pd.DataFrame(
    d_segment,
    index=rs.info['ch_names'],
    columns=np.round(times, 2)
)
print(df_segment)

print("\n=== Estadísticas por canal ===")
print(df_stats)

print(f"\nCanales después de drop_channels: {drop_names}")
print(f"Forma de datos tras crop: {crop_shape}")
print(f"Canales tras pick: {pick_names}")

#9. Probar plot 
#rs.plot(picks=['Chan0','Chan2'], start=0.0, duration=2.0, show_anotaciones=False)
#plt.show()


=== Datos del segmento (0.5s–1.5s) ===
        0.50   0.52   0.54   0.56   0.58   0.60   0.62   0.64   0.66   0.68  \
Chan0   25.0   26.0   27.0   28.0   29.0   30.0   31.0   32.0   33.0   34.0   
Chan1  125.0  126.0  127.0  128.0  129.0  130.0  131.0  132.0  133.0  134.0   
Chan2  225.0  226.0  227.0  228.0  229.0  230.0  231.0  232.0  233.0  234.0   
Chan3  325.0  326.0  327.0  328.0  329.0  330.0  331.0  332.0  333.0  334.0   

       ...   1.30   1.32   1.34   1.36   1.38   1.40   1.42   1.44   1.46  \
Chan0  ...   65.0   66.0   67.0   68.0   69.0   70.0   71.0   72.0   73.0   
Chan1  ...  165.0  166.0  167.0  168.0  169.0  170.0  171.0  172.0  173.0   
Chan2  ...  265.0  266.0  267.0  268.0  269.0  270.0  271.0  272.0  273.0   
Chan3  ...  365.0  366.0  367.0  368.0  369.0  370.0  371.0  372.0  373.0   

        1.48  
Chan0   74.0  
Chan1  174.0  
Chan2  274.0  
Chan3  374.0  

[4 rows x 50 columns]

=== Estadísticas por canal ===
    name     type    min      Q1  mediana      Q3